In [1]:
from dataclasses import replace
import numpy as np
import os
import torch
import matplotlib.pyplot as plt
import os.path
import sys
import logging
import pickle
from collections import defaultdict
from sklearn.linear_model import LinearRegression
from models.garnet_trigger import GNNGraphClassifier
from numpy.linalg import inv
import sklearn.metrics as metrics
from datasets import get_data_loaders
from tqdm.notebook import tqdm
from datasets.hit_graph_trigger_pileup import load_graph

In [2]:
DEVICE = "cuda:0"

In [3]:
# create model and load checkpoint
#model_result_folder = '/home1/giorgian/projects/trigger-detection-pipeline/sPHENIX/trigger_results/agnn/agnn-lr8.19806576478371e-05-b12-d71-PReLU-gi1-ln-True-n1600000/experiment_2024-04-26_13:41:37'
#MLP Trigger Layerwise
model_result_folder = '/home1/giorgian/projects/trigger-detection-pipeline/sPHENIX/trigger_results/agnn/agnn-lr8.19806576478371e-05-b12-d71-PReLU-gi1-ln-True-n1600000/experiment_2024-04-26_13:41:37'
config_file = model_result_folder + '/config.pkl'
config = pickle.load(open(config_file, 'rb'))
data_config = config.get('data')
dphi_max, dz_max = data_config['phi_slope_max'], data_config['z0_max']

model_config = config.get('model', {})
model_config.pop('loss_func')
model_config.pop('name')
model = GNNGraphClassifier(**model_config).to(DEVICE)

def load_checkpoint(checkpoint_file, model, optimizer=None):
    checkpoint = torch.load(checkpoint_file, map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['model'])
    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer'])
        return model, optimizer
    return model

# load_checkpoint
checkpoint_dir = os.path.join(model_result_folder, 'checkpoints')
checkpoint_file = sorted([os.path.join(checkpoint_dir, f) for f in os.listdir(checkpoint_dir) if f.startswith('model_checkpoint')])
checkpoint_file = checkpoint_file[-1]
print(checkpoint_file)
model = load_checkpoint(checkpoint_file, model)
print('Successfully reloaded!')

/home1/giorgian/projects/trigger-detection-pipeline/sPHENIX/trigger_results/agnn/agnn-lr8.19806576478371e-05-b12-d71-PReLU-gi1-ln-True-n1600000/experiment_2024-04-26_13:41:37/checkpoints/model_checkpoint_011.pth.tar
Successfully reloaded!


/tmp/ipykernel_915285/7784767.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_file, map_location=torch.device('cpu'))


In [4]:
data_config['use_intt'] = False
data_config

{'batch_size': 12,
 'construct_edges': False,
 'input_dir': '/ssd3/giorgian/hits-data-march-2024/trigger/1/',
 'input_dir2': '/ssd2/giorgian/hits-data-august-2022/nontrigger/0/',
 'load_full_event': False,
 'n_folders': 2,
 'n_mix': 10,
 'n_train': 800000,
 'n_valid': 175000,
 'n_workers': 16,
 'name': 'hit_graph_trigger_pileup',
 'phi_slope_max': 0.012193355583173944,
 'real_weight': 1,
 'z0_max': 14.220353082111805,
 'use_intt': False}

In [6]:
train_data_loader, valid_data_loader = get_data_loaders(distributed=False, rank=0, n_ranks=0, **data_config)

In [7]:
self = train_data_loader.dataset.dataset

In [8]:
ti = iter(train_data_loader)

In [9]:
batch = next(ti)

In [10]:
negatives = self.get_negatives(batch)

In [11]:
positives = self.get_positives(batch)

In [12]:
positives.active_node.shape, positives.x.shape

(torch.Size([4831]), torch.Size([4831, 5]))

In [13]:
batch.active_node.shape, batch.x.shape

(torch.Size([4918]), torch.Size([4918, 5]))

In [14]:
negatives.active_node.shape, negatives.x.shape

(torch.Size([4646]), torch.Size([4646, 5]))

In [34]:
batch.x = batch.x.to(torch.float)

In [15]:
anchor = model.encode(batch.to(DEVICE))

/home1/giorgian/projects/trigger-detection-pipeline/sPHENIX/tracking-GNN/models/garnet_trigger.py:132: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at /opt/conda/conda-bld/pytorch_1724789560443/work/torch/csrc/tensor/python_tensor.cpp:78.)
  temp = torch.cuda.FloatTensor(x.shape[0]).fill_(1)


In [16]:
positives = model.encode(positives.to(DEVICE))

RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float

In [ ]:
negatives = model.encode(negatives.to(DEVICE))

AttributeError: 'Tensor' object has no attribute 'active_node'

In [25]:
c = torch.nn.TripletMarginLoss()

In [29]:
c(anchor, positives, negatives)

tensor(1.0001, device='cuda:0', grad_fn=<MeanBackward0>)

In [28]:
len(batch.active_node[batch.active_node.to(bool)])

603

In [29]:
len(batch.active_node)

4771

In [39]:
active_nodes = batch.active_node.to(bool)
inactive_nodes = ~active_nodes

# Identify the batch indices and triggers for active nodes
active_batch_indices = torch.unique(batch.batch[active_nodes], sorted=False)
active_batch_triggers = batch.trigger[active_batch_indices]

# Collect the node features and batch indices of the inactive nodes
updated_node_features = batch.x[inactive_nodes]
updated_batch_indices = batch.batch[inactive_nodes]
num_inactive_nodes = updated_node_features.shape[0]

new_node_features_list = []
new_batch_indices_list = []

# Determine filenames based on trigger type
trigger_event_filenames = self.filenames_1 if self.f1_type else self.filenames_2
non_trigger_event_filenames = self.filenames_2 if self.f1_type else self.filenames_1

# Load new node features for each active batch index
for batch_index_tensor, batch_trigger_value in zip(active_batch_indices, active_batch_triggers):
    batch_index = batch_index_tensor.item()
    print(f'{batch_index=} {batch_trigger_value=}')
    if batch_trigger_value == 0:
        filename = np.random.choice(non_trigger_event_filenames)
    else:
        filename = np.random.choice(trigger_event_filenames)
    
    new_node_features, edge_index, y, event_info = load_graph(
        filename,
        self.cylindrical_features_scale,
        self.phi_slope_max,
        self.z0_max,
        use_intt=self.use_intt,
        construct_edges=self.construct_edges,
        drop_l1=self.drop_l1,
        drop_l2=self.drop_l2,
        drop_l3=self.drop_l3,
        add_global_node=self.add_global_node
    )
    new_node_features_list.append(new_node_features)
    new_batch_indices_list.append(np.ones(new_node_features.shape[0], dtype=int) * batch_index)
    
# Concatenate new node features and batch indices
new_node_features_array = np.concatenate(new_node_features_list, axis=0)
num_new_nodes = new_node_features_array.shape[0]

updated_node_features = torch.cat(
    [updated_node_features, torch.tensor(new_node_features_array).to(updated_node_features.device)],
    dim=0
)
new_batch_indices_array = np.concatenate(new_batch_indices_list, axis=0)
updated_batch_indices = torch.cat(
    [updated_batch_indices, torch.tensor(new_batch_indices_array).to(updated_batch_indices.device)],
    dim=0
)

# Update active_nodes: new nodes are active
updated_active_nodes = torch.cat(
    [torch.zeros(num_inactive_nodes, dtype=torch.bool),
     torch.ones(num_new_nodes, dtype=torch.bool)],
    dim=0
).to(active_nodes.device)

# Assign the updated tensors back to the batch object
batch.x = updated_node_features
batch.batch = updated_batch_indices
batch.active_node = updated_active_nodes


batch_index=0 batch_trigger_value=tensor(False)
batch_index=1 batch_trigger_value=tensor(False)
batch_index=2 batch_trigger_value=tensor(False)
batch_index=3 batch_trigger_value=tensor(False)
batch_index=4 batch_trigger_value=tensor(True)
batch_index=5 batch_trigger_value=tensor(False)
batch_index=6 batch_trigger_value=tensor(True)
batch_index=7 batch_trigger_value=tensor(True)
batch_index=8 batch_trigger_value=tensor(True)
batch_index=9 batch_trigger_value=tensor(False)
batch_index=10 batch_trigger_value=tensor(False)
batch_index=11 batch_trigger_value=tensor(False)


In [41]:
active_nodes = batch.active_node.to(bool)
inactive_nodes = ~active_nodes

# Identify the batch indices and triggers for active nodes
active_batch_indices = torch.unique(batch.batch[active_nodes], sorted=False)
active_batch_triggers = batch.trigger[active_batch_indices]

# Collect the node features and batch indices of the inactive nodes
updated_node_features = batch.x[inactive_nodes]
updated_batch_indices = batch.batch[inactive_nodes]
num_inactive_nodes = updated_node_features.shape[0]

new_node_features_list = []
new_batch_indices_list = []

# Determine filenames based on trigger type
trigger_event_filenames = self.filenames_1 if self.f1_type else self.filenames_2
non_trigger_event_filenames = self.filenames_2 if self.f1_type else self.filenames_1

# Load new node features for each active batch index
for batch_index_tensor, batch_trigger_value in zip(active_batch_indices, active_batch_triggers):
    batch_index = batch_index_tensor.item()
    print(f'{batch_index=} {batch_trigger_value=}')
    if batch_trigger_value == 0:
        filename = np.random.choice(trigger_event_filenames)
    else:
        filename = np.random.choice(non_trigger_event_filenames)
    
    new_node_features, edge_index, y, event_info = load_graph(
        filename,
        self.cylindrical_features_scale,
        self.phi_slope_max,
        self.z0_max,
        use_intt=self.use_intt,
        construct_edges=self.construct_edges,
        drop_l1=self.drop_l1,
        drop_l2=self.drop_l2,
        drop_l3=self.drop_l3,
        add_global_node=self.add_global_node
    )
    new_node_features_list.append(new_node_features)
    new_batch_indices_list.append(np.ones(new_node_features.shape[0], dtype=int) * batch_index)
    
# Concatenate new node features and batch indices
new_node_features_array = np.concatenate(new_node_features_list, axis=0)
num_new_nodes = new_node_features_array.shape[0]

updated_node_features = torch.cat(
    [updated_node_features, torch.tensor(new_node_features_array).to(updated_node_features.device)],
    dim=0
)
new_batch_indices_array = np.concatenate(new_batch_indices_list, axis=0)
updated_batch_indices = torch.cat(
    [updated_batch_indices, torch.tensor(new_batch_indices_array).to(updated_batch_indices.device)],
    dim=0
)

# Update active_nodes: new nodes are active
updated_active_nodes = torch.cat(
    [torch.zeros(num_inactive_nodes, dtype=torch.bool),
     torch.ones(num_new_nodes, dtype=torch.bool)],
    dim=0
).to(active_nodes.device)

# Assign the updated tensors back to the batch object
batch.x = updated_node_features
batch.batch = updated_batch_indices
batch.active_node = updated_active_nodes
batch.trigger = ~batch.trigger


batch_index=0 batch_trigger_value=tensor(True)
batch_index=1 batch_trigger_value=tensor(True)
batch_index=2 batch_trigger_value=tensor(True)
batch_index=3 batch_trigger_value=tensor(True)
batch_index=4 batch_trigger_value=tensor(False)
batch_index=5 batch_trigger_value=tensor(True)
batch_index=6 batch_trigger_value=tensor(False)
batch_index=7 batch_trigger_value=tensor(False)
batch_index=8 batch_trigger_value=tensor(False)
batch_index=9 batch_trigger_value=tensor(True)
batch_index=10 batch_trigger_value=tensor(True)
batch_index=11 batch_trigger_value=tensor(True)


In [32]:
batch_index=0 batch_trigger_value=tensor(False)
batch_index=1 batch_trigger_value=tensor(False)
batch_index=2 batch_trigger_value=tensor(False)
batch_index=3 batch_trigger_value=tensor(False)
batch_index=4 batch_trigger_value=tensor(True)
batch_index=5 batch_trigger_value=tensor(False)
batch_index=6 batch_trigger_value=tensor(True)
batch_index=7 batch_trigger_value=tensor(True)
batch_index=8 batch_trigger_value=tensor(True)
batch_index=9 batch_trigger_value=tensor(False)
batch_index=10 batch_trigger_value=tensor(False)
batch_index=11 batch_trigger_value=tensor(False)

tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.],
       dtype=torch.float64)

In [30]:
batch.active_node.shape

torch.Size([4781])

batch_index=0 batch_trigger_value=tensor(False)
batch_index=1 batch_trigger_value=tensor(False)
batch_index=2 batch_trigger_value=tensor(True)
batch_index=3 batch_trigger_value=tensor(True)
batch_index=4 batch_trigger_value=tensor(False)
batch_index=5 batch_trigger_value=tensor(False)
batch_index=6 batch_trigger_value=tensor(True)
batch_index=7 batch_trigger_value=tensor(False)
batch_index=8 batch_trigger_value=tensor(True)
batch_index=9 batch_trigger_value=tensor(True)
batch_index=10 batch_trigger_value=tensor(False)
batch_index=11 batch_trigger_value=tensor(False)


In [25]:
updated_active_nodes

tensor([False, False, False,  ...,  True,  True,  True])

In [17]:
new_batch, new_batches

(tensor([ 0,  0,  0,  ..., 11, 11, 11]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,
         1.,  1.,  1.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
         2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
         2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
         2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
         2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
         2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
         2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
         2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
         2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
         2

tensor([ 0.,  0.,  0.,  ..., 11., 11., 11.], dtype=torch.float64)

In [14]:
dir(batch)

['__annotations__',
 '__call__',
 '__cat_dim__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__inc__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_edge_attr_cls',
 '_edge_to_layout',
 '_edges_to_layout',
 '_get_edge_index',
 '_get_tensor',
 '_get_tensor_size',
 '_multi_get_tensor',
 '_put_edge_index',
 '_put_tensor',
 '_remove_edge_index',
 '_remove_tensor',
 '_store',
 '_tensor_attr_cls',
 '_to_type',
 'apply',
 'apply_',
 'batch',
 'batch_size',
 'clone',
 'coalesce',
 'concat',
 'contains_isolated_nodes',
 'contains_self_loops',
 'con

In [12]:
new_x.shape

(553, 5)

In [13]:
new_batch

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2

In [40]:
i

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [42]:
batch.trigger

tensor([ True, False,  True,  True,  True,  True, False,  True, False, False,
         True, False])